In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import cv2
import numpy as np
import os
import pandas as pd
from IPython.display import display


In [6]:
# ===============================
# مسیرها
# ===============================
model_dir = "/content/drive/MyDrive/COVID_Model"
image_dir = "/content"

# ترتیب کلاس‌ها
labels = ['Normal', 'COVID-19', 'Pneumonia']


In [7]:
# ===============================
# ریست و سشن
# ===============================
tf.reset_default_graph()
sess = tf.Session()

# ===============================
# لود مدل
# ===============================
meta_path = os.path.join(model_dir, "model .meta")
saver = tf.train.import_meta_graph(meta_path)
saver.restore(sess, os.path.join(model_dir, "model"))
graph = tf.get_default_graph()

# ===============================
# پیدا کردن ورودی‌ها و خروجی
# ===============================
input_256 = graph.get_tensor_by_name("input_1:0")
input_480 = graph.get_tensor_by_name("input_2:0")
softmax_tensor = [op.outputs[0] for op in graph.get_operations() if "Softmax" in op.name][-1]


In [8]:
# ===============================
# پردازش تصاویر
# ===============================
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png','.jpg','.jpeg'))]

results = []
correct = 0

for img_name in image_files:
    img_path = os.path.join(image_dir, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    # -------- لیبل واقعی از اسم فایل --------
    name = img_name.lower()
    if "covid" in name:
        actual = "COVID-19"
    elif "pneumonia" in name:
        actual = "Pneumonia"
    elif "normal" in name:
        actual = "Normal"
    else:
        actual = "Unknown"

    # -------- پیش‌پردازش --------
    gray_256 = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (256,256))
    gray_256 = gray_256.astype(np.float32)/255.0
    gray_256 = np.expand_dims(gray_256, axis=(0,-1))

    rgb_480 = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (480,480))
    rgb_480 = rgb_480.astype(np.float32)/255.0
    rgb_480 = np.expand_dims(rgb_480, axis=0)

    # -------- اجرای مدل --------
    preds = sess.run(
        softmax_tensor,
        feed_dict={
            input_256: gray_256,
            input_480: rgb_480
        }
    )[0]

    pred_label = labels[np.argmax(preds)]
    confidence = np.max(preds) * 100

    is_correct = (pred_label == actual)
    if is_correct:
        correct += 1

    results.append({
        "نام فایل": img_name,
        "جواب واقعی": actual,
        "تشخیص مدل": pred_label,
        "اطمینان": f"{confidence:.2f}%",
        "وضعیت": "✅ درست" if is_correct else "❌ غلط"
    })


In [9]:
# ===============================
# نمایش نتایج
# ===============================
df = pd.DataFrame(results)
display(df)

accuracy = (correct / len(results)) * 100
print("\n" + "="*30)
print(f"📊 دقت کل مدل: {accuracy:.2f}%")
print(f"✅ درست: {correct}")
print(f"❌ غلط: {len(results)-correct}")
print("="*30)


,نام فایل,جواب واقعی,تشخیص مدل,اطمینان,وضعیت
0,COVID-1024.png,COVID-19,COVID-19,92.24%,✅ درست
1,Viral_20Pneumonia-10.png,Pneumonia,COVID-19,99.76%,❌ غلط
2,COVID-10.png,COVID-19,COVID-19,98.91%,✅ درست
3,COVID-1023.png,COVID-19,COVID-19,99.20%,✅ درست
4,Normal-10022.png,Normal,Normal,99.74%,✅ درست
5,Normal-10008.png,Normal,COVID-19,95.46%,❌ غلط
6,Viral_20Pneumonia-100.png,Pneumonia,COVID-19,99.95%,❌ غلط
7,Normal-10020.png,Normal,Normal,92.81%,✅ درست
8,COVID-1.png,COVID-19,COVID-19,90.02%,✅ درست
9,Normal-1.png,Normal,Normal,99.84%,✅ درست



📊 دقت کل مدل: 75.00%
✅ درست: 15
❌ غلط: 5
